In [1]:
import tensorflow as tf
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [2]:
import numpy as np

In [3]:
class gl:
    myftype = 'float64'
    myitype = 'int64'
    k = tf.constant(0.1, dtype = myftype)
    M = tf.constant(50, dtype = myitype)
    deltat = tf.constant(0.1, dtype = myftype)
    numsteps = 10
    h = tf.constant(0.5, dtype = myftype)
    grid = tf.multiply(k,tf.cast(tf.range(tf.negative(M), tf.add(M, 1), delta = 1), dtype = myftype))
    gridx, gridy = tf.meshgrid(grid, grid, indexing = 'ij')
    gridsize = tf.add(tf.multiply(M, 2), 1)
    gridsize2 = tf.pow(gridsize, 2)

In [4]:
def tff(theta, x):
    fval = tf.multiply(theta[0],tf.subtract(theta[1],x))
    return fval

def tfg(theta, x):
    gval = tf.multiply(theta[2],tf.ones(tf.shape(x),dtype=gl.myftype))
    return gval

In [5]:
# def integrandmat(inx, iny, th):
#     my2 = tf.constant(2.0,gl.myftype)
#     tfmu = tf.add(iny,tf.multiply(tff(theta=th,x=iny),gl.h))
#     tfsig = tf.multiply(tf.sqrt(gl.h),tfg(theta=th,x=iny))
#     tfc0 = tf.reciprocal(tf.multiply(tf.sqrt(tf.multiply(my2,tf.constant(np.pi,dtype=gl.myftype))),tfsig))
#     tfnumer = tf.negative(tf.square(tf.subtract(inx,tfmu)))
#     tfdenom = tf.multiply(my2,tf.square(tfsig))
#     tfprop = tf.multiply(tfc0,tf.exp(tf.divide(tfnumer,tfdenom)))
#     return tfprop

In [6]:
def createAB(theta):
    AMtheta = tf.constant(0.5, dtype = gl.myftype)
    omtheta = tf.subtract(tf.constant(1.0, dtype = gl.myftype), AMtheta)
    alpha = []
    alpha.append(tf.div(tf.constant(0.5, dtype = gl.myftype), tf.multiply(AMtheta, omtheta)))
    alpha.append(tf.multiply(alpha[0], tf.add(tf.square(AMtheta), tf.square(omtheta))))
    
    outAval = tf.zeros(shape = 1, dtype = gl.myftype)
    outAind = tf.zeros(shape = (1,3), dtype = gl.myitype)
    Aval = [outAval]
    Aind = [outAind]
    
    fvec = f(theta, grid)
    gvec = g(theta, grid)
    
    g2m1 = tf.square(gvec)
    col1 = tf.ones(shape = (gl.gridsize, 1), dtype = gl.myitype)
    colr = tf.reshape(tf.cast(tf.range(0, gl.gridsize, delta = 1), dtype = gl.myitype), (gl.gridsize, 1))
    
    for i in range(gl.gridsize):
        for j in range(gl.gridsize):
            if (tf.abs(tf.subtract(i,j)) > np.inf):
                continue

            yi = grid[i]
            ystar = grid[j]
            mv = tf.add(ystar, tf.multiply(tf.subtract((tf.multiply(alpha[0], f(th, ystar)), tf.multiply(alpha[1], fvec))), tf.multiply(omtheta, gl.h)))
            g2star = tf.square(g(th, ystar))
            vv = tf.maximum(0, alpha[0]*g2star - alpha[1]*g2m1)*omtheta*gl.h
                        
            # the original code used scale = np.sqrt(vv) but scale is \sigma^2, here it's \sigma
            # confirm which one has to be used!
            thesepdfs = tf.contrib.distributions.Normal(mu = mv, sigma = tf.sqrt(vv))
            thesevals = thesepdfs.pdf(yi)
            
            indblock = tf.concat([i*col1,j*col1,colr], axis = 1)
            Aind.append(indblock)
            Aval.append(thesevals)
    
    # np.hstack(vec) == tf.concat(vec, axis = 1)
    # np.vstack(vec) == tf.concat(vec, axis = 0)
    outAind = tf.concat(Aind[1:], axis = 0)
    outAval = tf.concat(Aval[1:], axis = 1)

    Bds = [gl.gridsize, gl.gridsize]
    Bind = tf.zeros((0,2), dtype = gl.myitype)
    thisvar = g2m1 * AMtheta * gl.h
    c0mod = tf.div(tf.constant(1., dtype = gl.myftype), tf.sqrt(tf.constant(2., dtype = gl.myftype)*np.pi*thisvar))
    propvals = tf.exp(tf.negative(AMtheta*gl.h/2.0)*tf.square(fvec)/g2m1) * c0mod
    Bval = [propvals]
    indblock = tf.concat([colr, colr], axis = 1)
    Bind = [tf.concat([Bind, indblock], axis = 0)]

    for curdiag in range(1, gl.gridsize):
        thislen = gl.gridsize - curdiag
        mymean = curdiag * gl.k + fvec * gl.h * AMtheta
        thisdiag = tf.exp(-tf.square(mymean)/(2.0*thisvar)) * c0mod
        thisdiag = thisdiag[-thislen:]
        indblock = tf.concat([colr[:(gl.gridsize - curdiag)], (colr[:(gl.gridsize - curdiag)] + curdiag)], axis = 1)
        Bind.append(indblock) 
        Bval.append(thisdiag) 

    for curdiag in range(1, gl.gridsize):
        thislen = gl.gridsize - curdiag
        mymean = -curdiag * gl.k + fvec * gl.h * AMtheta
        thisdiag = tf.exp(-tf.square(mymean)/(2.0*thisvar)) * c0mod
        thisdiag = thisdiag[:thislen]
        indblock = tf.concat([(colr[:(gl.gridsize - curdiag)] + curdiag), colr[:(gl.gridsize - curdiag)]], axis = 1)
        Bind.append(indblock) 
        Bval.append(thisdiag) 

    outBind = tf.concat(Bind, axis = 0)
    outBval = tf.concat(Bval, axis = 1)
    
    return outAind, outAval, outBind, outBval

In [7]:
def computeAMprop(Aind, Aval, Bden):
    sortAind = np.argsort(Aind[:,2])
    newAind = Aind[sortAind,:]
    newAval = tf.array(Aval)
    newAval = newAval[sortAind]
    mypropcols = []
    startind = 0
    thisds = [gl.gridsize, gl.gridsize]
    for j in range(gl.gridsize):
        if j == (gl.gridsize-1):
            endind = len(Aval)
        else:
            endind = np.searchsorted(newAind[:,2],(j+1))

        thesevals = newAval[startind:endind]
        theseinds = newAind[startind:endind,0:2]
        thisten = tf.SparseTensor(indices = theseinds, values = thesevals, dense_shape = thisds)
        newcolumn = (gl.k)*tf.sparse_tensor_dense_matmul(thisten, tf.slice(Bden, [0,j], [gl.gridsize,1]))
        mypropcols.append(newcolumn[:,0])
        startind = endind

    myprop = tf.stack(mypropcols)
    return myprop

In [ ]:
def integrandmat():
    Aind, Aval, Bind, Bval = createAB(tol = np.inf)
    Bds = [gl.gridsize, gl.gridsize]
    Bten = tf.SparseTensor(indices = Bind, values = Bval, dense_shape = Bds)
    Bden = tf.sparse_tensor_to_dense(Bten, validate_indices = False)
    prop = computeAMprop(Aind, Aval, Bden)

In [8]:
def dtqall(theta, init, final):
    # supposed to be a column vector
    lamb = tf.expand_dims(integrandmat(gl.grid, init, theta),1)
    
    # supposed to be a row vector
    gamm = gl.k * tf.expand_dims(integrandmat(final, gl.grid, theta),0)
    
    # declare two tensors in advance
    
    lamblist = []
    gammlist = []
    lamblist.append(lamb)
    gammlist.append(gamm)
    
    for j in range(gl.numsteps-2):
        lamblist.append(gl.k*tf.matmul(gl.A,lamblist[j]))
        gammlist.append(gl.k*tf.matmul(gammlist[j],gl.A))
        
    complete = tf.reshape(tf.matmul(gammlist[0],lamblist[gl.numsteps-2]),shape=[])
    
    # both first and last wil be column vectors
    c = tf.multiply(gl.k,complete)
    first = tf.divide(tf.multiply(tf.transpose(gammlist[gl.numsteps-2]),lamblist[0]),c)
    last = tf.divide(tf.multiply(tf.transpose(gammlist[0]),lamblist[gl.numsteps-2]),c)

    # compute all intermediate 2d pdfs
    del lamblist[-1]
    del gammlist[-1]
    lambtensor = tf.stack(lamblist)
    # diag = []
    # diag.append(tf.shape(lambtensor))
    gammtensor = tf.stack(gammlist[::-1])
    # diag.append(tf.shape(gammtensor))
    A0 = tf.expand_dims(gl.A,0)
    # diag.append(tf.shape(A0))
    pdf2dlist = tf.transpose(tf.multiply(gammtensor, lambtensor),perm=[0,2,1])
    # diag.append(tf.shape(pdf2dlist))
    pdf2dlist = tf.multiply(pdf2dlist, A0)
    pdf2dlist = tf.divide(pdf2dlist, c)

    return complete, first, last, pdf2dlist #, diag


In [9]:
def logG(x, y, theta):
    fv = tff(theta,y)
    gv = tfg(theta,y)
    mu = tf.add(y,tf.multiply(fv,gl.h))
    pr = tf.subtract(x,mu)
    pr2 = tf.square(pr)
    gv2 = tf.square(gv)
    my2 = tf.constant(2.0,dtype=gl.myftype)
    mypi = tf.constant(np.pi,dtype=gl.myftype)
    lgp1 = tf.negative(tf.divide(tf.log(tf.multiply(my2*mypi*gl.h,gv2)),my2))
    lgp2 = tf.negative(tf.divide(pr2,tf.multiply(my2*gl.h,gv2)))
    lg = tf.add(lgp1,lgp2)        
    return lg

In [10]:
# allout is the result of a call to dtqall
# here x is the data
def qfun(theta, allout, init, final):
    q = []

    # first term in the summation (j=1 case)
    part1 = logG(gl.grid,init,theta)
    q.append(tf.reduce_sum(tf.multiply(part1,allout[1][:,0]))*gl.k)

    # last term in the summation (j=F case)
    part2 = logG(final,gl.grid,theta)
    q.append(tf.reduce_sum(tf.multiply(part2,allout[2][:,0]))*gl.k)
    
    # all intermediate terms
    part3 = logG(gl.gridx,gl.gridy,theta)
    # for j in range(gl.numsteps-2):
    #     q.append(tf.tensordot(part3,allout[3][j,:],axes=[[0,1],[0,1]])*gl.k*gl.k)
    # test = tf.add_n(test)
    
    q.append(tf.reduce_sum(tf.multiply(tf.expand_dims(part3,0),allout[3]))*gl.k*gl.k)
    
    qout = tf.negative(tf.add_n(q))

    return qout


In [11]:
thval = [0.5,0.7,0.2]
thetak = tf.constant(thval,dtype=gl.myftype)
init = tf.constant(0.01,dtype=gl.myftype)
final = tf.constant(0.2,dtype=gl.myftype)
gl.A = integrandmat(gl.gridx, gl.gridy, thetak)
allout = dtqall(thetak, init, final)
# all of the above completes the E step

# print(sess.run(qfun(th, allout, init, final), feed_dict = {th : thval}))
# print(sess.run(tf.gradients(qfun(th, allout, init, final),th), feed_dict = {th : thval}))
# part3 = logG(gl.gridx,gl.gridy,th)
# print(sess.run(tf.shape(allout[3][0,:]), feed_dict = {th : thval}))
# print(sess.run(tf.tensordot(part3,allout[3][0,:],axes=[[0,1],[0,1]]), feed_dict = {th : thval}))
# print(sess.run(fir, feed_dict = {th : thval}))
# print(sess.run(gl.k*gl.k*tf.reduce_sum(pdf,axis=[1,2]), feed_dict = {th : thval}))
# print(sess.run(tf.gradients(logG(gl.gridx,gl.gridy,th),th),  feed_dict = {th : thval}))

TypeError: Fetch argument array([0]) has invalid type <class 'numpy.ndarray'>, must be a string or Tensor. (Can not convert a ndarray into a Tensor or Operation.)

In [ ]:
th = tf.Variable(thetak)
sess.run(tf.global_variables_initializer())
loss = qfun(th, allout, init, final)
learning_rate = 0.05
my_opt = tf.train.GradientDescentOptimizer(learning_rate)
train_step = my_opt.minimize(loss)

In [ ]:
sess.run(train_step)

In [ ]:
print(sess.run(th))

In [ ]:
q = tf.Variable(np.array([[2.0,-3.0],[-1.5,4.5]]))
inc = tf.constant(np.array([[0.0,1.0],[0.0,0.0]]))
sess.run(tf.global_variables_initializer())
print(sess.run(q))
tf.assign_add(q, inc)
print(sess.run(q))